In [197]:
import numpy as np

data = open("small.in", "r").read()

data = data.split()
R = int(data[0])
C = int(data[1])
L = int(data[2])  # at least L cells of each ingredient
H = int(data[3])  # at most H cells in total
pizza = np.array([[0 if ingredient == "T" else 1 for ingredient in row] for row in data[4:]], dtype=np.int8)

In [198]:
R, C, L, H

(6, 7, 1, 5)

In [199]:
# enumerate all possible sizes
# min == 2*L  max == H
from sympy import divisors

sizes = []

for n_cells in range(2 * L, H + 1):
    for div in divisors(n_cells):
        sizes.append((div, n_cells // div))

In [200]:
len(sizes)

9

In [201]:
# enumerate all subsets
subsets = []
for rows, cols in sizes:
    for i in range(pizza.shape[0] - rows + 1):
        for j in range(pizza.shape[1] - cols + 1):
            toppings = pizza[i:i+rows, j:j+cols].sum()  
            
            if toppings >= L and rows*cols - toppings >= L:
                subsets.append((i, j, i+rows, j+cols))
                
subsets_idx = ["%d_%d_%d_%d" % (a, b, c, d) for a, b, c, d in subsets]

In [202]:
len(subsets)

149

In [203]:
# constraints
constraints = {}

for i, s in enumerate(subsets):
    for c_i in range(s[0], s[2]):
        for c_j in range(s[1], s[3]):
            if (c_i, c_j) not in constraints:
                constraints[(c_i, c_j)] = []
            constraints[(c_i, c_j)].append(i)    
      
# chop off singletons
constraints = {k:v for k, v in constraints.items() if len(v) > 1}

In [204]:
sum([len(cs) for cs in constraints.values()])

533

In [205]:
import pulp
from pulp import LpProblem, LpVariable, LpMaximize

problem = LpProblem(name="pizza", sense=LpMaximize)

In [206]:
# variables
slices = LpVariable.dicts("slices", subsets_idx, lowBound=0, upBound=1, cat="Binary")

In [207]:
# objective
from pulp import LpAffineExpression

exp = LpAffineExpression()
for i, s in enumerate(subsets):
    exp += (s[2]-s[0])*(s[3]-s[1]) * slices[subsets_idx[i]]
    
problem += exp

In [208]:
# constraints
for pos, ids in constraints.items():
    exp = LpAffineExpression()
    for i in ids:
        exp += slices[subsets_idx[i]]
        
    problem += exp <= 1

In [209]:
# solve
print("Solving...")
problem.solve(pulp.solvers.GLPK())

Solving...


1

In [210]:
problem.objective.value()

42

In [211]:
for i, s in enumerate(subsets):
    if slices[subsets_idx[i]].value() > 0.0:
        print(s, (s[2]-s[0])*(s[3]-s[1]))

(0, 0, 1, 2) 2
(5, 2, 6, 7) 5
(0, 2, 5, 3) 5
(0, 3, 5, 4) 5
(0, 4, 5, 5) 5
(0, 5, 5, 6) 5
(0, 6, 5, 7) 5
(1, 0, 6, 1) 5
(1, 1, 6, 2) 5
